<a href="https://colab.research.google.com/github/neeluvermaiitj/Dependable-AI/blob/main/D_Detection_on_Cipher10_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install adversarial-robustness-toolbox

# Perform detection using new CNN model

In [ ]:
#from  art.defences.detector.evasion  import BinaryInputDetector
#from art.estimators.classification import KerasClassifier
import keras
from keras.datasets import cifar10
import tensorflow as tf
from keras.models import load_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization
from keras.optimizers import SGD
import torch
import numpy as np
import scipy.io as sio
from scipy.io import loadmat

In [ ]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [ ]:
# define cnn model
def define_detector_model():
  model = Sequential()
  model.add(Conv2D(input_shape=(32, 32, 3),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  #model.BatchNormalization(),
  model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  # compile model
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  #model.compile(optimizer=opt, loss='binary_categorical', metrics=['accuracy'])
  return model

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm 
def load_dataset():
  # load dataset
  (trainX, trainY), (testX, testY) = cifar10.load_data()
  # one hot encode target values
  trainY = to_categorical(trainY)
  testY = to_categorical(testY)
  return trainX, trainY,testX, testY
def run_test_harness():
  MAT_save_name = 'trainXSVM_complete_data.mat'
  path = F"/content/gdrive/My Drive/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  trainXSVM = annots['mydata']

  MAT_save_name = 'trainYSVM_complete_data.mat'
  path = F"/content/gdrive/My Drive/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  trainYSVM = annots['mydata']

  MAT_save_name = 'testXSVM_complete_data.mat'
  path = F"/content/gdrive/My Drive/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  testXSVM = annots['mydata']

  MAT_save_name = 'testYSVM_complete_data.mat'
  path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  testYSVM = annots['mydata']

  trainYSVM = trainYSVM.reshape(150000,1)
  trainXSVM, trainYSVM = shuffle_in_unison(trainXSVM, trainYSVM)
  testXSVM,testYSVM = shuffle_in_unison(testXSVM, testYSVM)

  #print(trainXSVM.shape , trainYSVM.shape , testXSVM.shape , testYSVM,shape)

  #print(error)
  trainXSVM = trainXSVM[:1000]
  trainYSVM = trainYSVM[:1000]

  testXSVM = trainXSVM[:100]
  testYSVM = testYSVM[:100]
  
  trainYSVM = to_categorical(trainYSVM)
  testYSVM = to_categorical(testYSVM)
  
  if load_trained_detector_model == 0:
    # define model
    detector_model = define_detector_model()
    detector_model.summary()
    # fit model

    history = detector_model.fit(trainXSVM, trainYSVM, epochs=10, batch_size=64, validation_data=(testXSVM, testYSVM), verbose=1)
    model_save_name = 'detector_model.h5'
    path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{model_save_name}" 
    detector_model.save(path)
  else:
    model_save_name = 'detector_model.h5'
    path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{model_save_name}"  
    detector_model = load_model(path)

  # evaluate model
  _, acc = detector_model.evaluate(testXSVM, testYSVM, verbose=1)
  print('> %.3f' % (acc * 100.0))

# entry point, run the test harness
labels = [  'airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
tf.compat.v1.disable_eager_execution()
loss_object = tf.keras.losses.CategoricalCrossentropy()
load_trained_detector_model=0
run_test_harness()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)       

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1000/1000 [==============================] - 2s 2ms/sample - loss: 0.7109 - accuracy: 0.5880 - val_loss: 0.6754 - val_accuracy: 0.7400
Epoch 2/10
1000/1000 [==============================] - 0s 443us/sample - loss: 0.6341 - accuracy: 0.6820 - val_loss: 0.6711 - val_accuracy: 0.7400
Epoch 3/10
1000/1000 [==============================] - 0s 451us/sample - loss: 0.5900 - accuracy: 0.6890 - val_loss: 0.6614 - val_accuracy: 0.7400
Epoch 4/10
1000/1000 [==============================] - 0s 493us/sample - loss: 0.5269 - accuracy: 0.7360 - val_loss: 0.6587 - val_accuracy: 0.7400
Epoch 5/10
1000/1000 [==============================] - 0s 448us/sample - loss: 0.4768 - accuracy: 0.7880 - val_loss: 0.6470 - val_accuracy: 0.7400
Epoch 6/10
1000/1000 [==============================] - 0s 465us/sample - loss: 0.3904 - accuracy: 0.8390 - val_loss: 0.6418 - val_accuracy: 0.7400
Epoch 7/10
1000/1000 [==============================] - 0s 448us/sample - loss: 0.3146 - accuracy: 0.8970 - val_loss: 0.6383 

# Perform detection using new CNN2 model

In [ ]:


#from  art.defences.detector.evasion  import BinaryInputDetector
#from art.estimators.classification import KerasClassifier
import keras
from keras.datasets import cifar10
import tensorflow as tf
from keras.models import load_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization
from keras.optimizers import SGD
import torch
import numpy as np
import scipy.io as sio
from scipy.io import loadmat

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b
# define cnn model

def define_detector_model():
  model = Sequential()
  model.add(Conv2D(input_shape=(32, 32, 3),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  #model.BatchNormalization(),
  model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  # compile model
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  #model.compile(optimizer=opt, loss='binary_categorical', metrics=['accuracy'])
  return model

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm 
def load_dataset():
  # load dataset
  (trainX, trainY), (testX, testY) = cifar10.load_data()
  # one hot encode target values
  trainY = to_categorical(trainY)
  testY = to_categorical(testY)
  return trainX, trainY,testX, testY
def run_test_harness():
  MAT_save_name = 'trainXSVM_complete_data.mat'
  path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  trainXSVM = annots['mydata']

  MAT_save_name = 'trainYSVM_complete_data.mat'
  path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  trainYSVM = annots['mydata']

  MAT_save_name = 'testXSVM_complete_data.mat'
  path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  testXSVM = annots['mydata']

  MAT_save_name = 'testYSVM_complete_data.mat'
  path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
  annots = loadmat(path)
  annots.keys()  
  testYSVM = annots['mydata']

  trainYSVM = trainYSVM.reshape(150000,1)
  trainXSVM, trainYSVM = shuffle_in_unison(trainXSVM, trainYSVM)
  testXSVM,testYSVM = shuffle_in_unison(testXSVM, testYSVM)
  
  trainYSVM = to_categorical(trainYSVM)
  testYSVM = to_categorical(testYSVM)
  
  if load_trained_detector_model == 0:
    # define model
    detector_model = define_detector_model()
    detector_model.summary()
    # fit model

    history = detector_model.fit(trainXSVM, trainYSVM, epochs=10, batch_size=64, validation_data=(testXSVM, testYSVM), verbose=1)
    model_save_name = 'detector_model.h5'
    path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{model_save_name}" 
    detector_model.save(path)
  else:
    model_save_name = 'detector_model.h5'
    path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{model_save_name}"  
    detector_model = load_model(path)

  # evaluate model
  _, acc = detector_model.evaluate(testXSVM, testYSVM, verbose=1)
  print('> %.3f' % (acc * 100.0))

# entry point, run the test harness
labels = [  'airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
tf.compat.v1.disable_eager_execution()
loss_object = tf.keras.losses.CategoricalCrossentropy()
load_trained_detector_model=1
run_test_harness()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


> 66.377
